In [5]:
import pandas as pd

read_df = pd.read_csv('data/FNSPID dataset/SNP5gb.csv')

C:\Users\Jay Tai\AppData\Local\Temp\ipykernel_10816\2924716566.py:3: DtypeWarning: Columns (2,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  read_df = pd.read_csv('data/FNSPID dataset/SNP5gb.csv')


In [6]:
selected_symbols = ['MSFT', 'NVDA', 'AAPL', 'AMZN', 'META', 'AVGO', 'BRK.B', 'TSLA', 'GOOG', 'GOOGL', 'WMT', 'JPM', 'LLY', 'V', 'ORCL', 'NFLX', 'XOM', 'MA', 'COST', 'PG']

read_df = read_df[read_df['Stock_symbol'].isin(selected_symbols)]

df = pd.DataFrame({
    'Date': read_df['Date'],
    'Title': read_df['Article_title'],
    'StockSymbol': read_df['Stock_symbol']
})

df['Date'] = pd.to_datetime(df['Date'], utc=True).dt.date

import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    if not isinstance(text, str) or text is None:
        return ""
    tokens = word_tokenize(text.lower())  # Tokenize and lowercase
    cleaned_tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha() and token not in stop_words]
    test_sentence = " ".join(cleaned_tokens)
    return test_sentence

df['Title'] = df['Title'].apply(preprocess)

from transformers import pipeline, set_seed
set_seed(999)

classifier = pipeline('sentiment-analysis', model='nickmuchi/distilroberta-finetuned-financial-text-classification')

[nltk_data] Downloading package wordnet to C:\Users\Jay
[nltk_data]     Tai\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Jay
[nltk_data]     Tai\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to C:\Users\Jay
[nltk_data]     Tai\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


config.json:   0%|          | 0.00/894 [00:00<?, ?B/s]

C:\Users\Jay Tai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Jay Tai\.cache\huggingface\hub\models--nickmuchi--distilroberta-finetuned-financial-text-classification. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  war

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/355 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cpu


In [8]:
predictions = classifier(df['Title'].tolist())

In [9]:
df['Sentiment'] = [
    pred['score'] if pred['label'] == 'bullish' else -pred['score'] if pred['label'] == 'bearish' else 0
    for pred in predictions
]

In [10]:
grouped_data = df.groupby('Date').agg({
    'Sentiment': 'sum'
})
grouped_data.index = pd.to_datetime(grouped_data.index)
grouped_data.to_csv('data/Versions/V2/V2 grp_df.csv')